Comenzamos con el dataset ya separado, para facilitar las cosas.

Instalamos y cargamos dependencias:

In [ ]:
install.packages("readr", version = "2.1.4")
install.packages("caret", version = "6.0-94")

In [1]:
library(readr)
library(caret)
library(dplyr)
library(ggplot2)
library(lattice)

Cargando paquete requerido: ggplot2

Cargando paquete requerido: lattice


Adjuntando el paquete: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




Cargamos los datasets:

In [2]:
df <- read_csv('https://raw.githubusercontent.com/gabichulas/ia-uncuyo-2024/refs/heads/main/tp7-ml/data/arbolado-mza-dataset.csv', show_col_types = FALSE) # nolint: line_length_linter.
train_df <- read_csv('https://raw.githubusercontent.com/gabichulas/ia-uncuyo-2024/refs/heads/main/tp7-ml/data/arbolado-mendoza-dataset-train.csv', show_col_types = FALSE)
val_df <- read_csv('https://raw.githubusercontent.com/gabichulas/ia-uncuyo-2024/refs/heads/main/tp7-ml/data/arbolado-mendoza-dataset-validation.csv', show_col_types = FALSE)

df <- as.data.frame(df)
train_df <- as.data.frame(train_df)
val_df <- as.data.frame(val_df)

print(val_df)

        id           especie ultima_modificacion               altura
1    37462            Morera    04/01/2011 12:01    Medio (4 - 8 mts)
2    20115            Morera    09/08/2011 12:38       Alto (> 8 mts)
3     8965            Morera    14/01/2011 10:27       Alto (> 8 mts)
4    43061           Paraiso    03/10/2011 10:04    Medio (4 - 8 mts)
5    13030            Morera    16/02/2011 09:49    Medio (4 - 8 mts)
6    37942              Acer    27/08/2010 10:50    Medio (4 - 8 mts)
7    18118           Paraiso    28/07/2011 09:44    Medio (4 - 8 mts)
8    13533            Morera    01/02/2011 12:37       Alto (> 8 mts)
9    17284            Morera    04/08/2011 13:22       Alto (> 8 mts)
10   41219    Fresno europeo    15/04/2011 13:57       Alto (> 8 mts)
11   22439           Paraiso    28/06/2011 09:35    Medio (4 - 8 mts)
12   35188  Fresno americano    29/03/2011 10:38       Alto (> 8 mts)
13    3107 Paraiso sombrilla    16/09/2011 11:05     Bajo (2 - 4 mts)
14   24546          

Creamos la función que genera la columna `prediction_prob` y la ejecutamos sobre nuestro dataframe:

In [3]:
add_prediction_prob <- function(data) {

  random_probs <- runif(nrow(data), min = 0, max = 1)
  data$prediction_prob <- random_probs
  return(data)
}

df <- add_prediction_prob(df)
val_df <- add_prediction_prob(val_df)
head(df)

,id,especie,ultima_modificacion,altura,circ_tronco_cm,diametro_tronco,long,lat,seccion,nombre_seccion,area_seccion,inclinacion_peligrosa,prediction_prob
,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>
1,0,Jacarand,06/05/2011 14:49,Medio (4 - 8 mts),60,Mediano,-68.84041,-32.87473,1,Parque Central,1655358,0,0.4862155
2,3,Fresno europeo,02/03/2012 13:26,Bajo (2 - 4 mts),52,Mediano,-68.85742,-32.89599,5,Residencial Sur,2423809,0,0.3591133
3,5,Morera,02/03/2012 13:23,Medio (4 - 8 mts),70,Grande,-68.85741,-32.89586,5,Residencial Sur,2423809,0,0.1924446
4,7,Morera,02/03/2012 13:19,Medio (4 - 8 mts),110,Grande,-68.85730,-32.89583,5,Residencial Sur,2423809,0,0.5653445
5,8,Morera,02/03/2012 13:13,Medio (4 - 8 mts),54,Mediano,-68.85722,-32.89584,5,Residencial Sur,2423809,0,0.2367850
6,10,Morera,02/03/2012 13:10,Medio (4 - 8 mts),100,Grande,-68.85708,-32.89585,5,Residencial Sur,2423809,0,0.8102634


Creamos la funcion `random_classifier` y la corremos sobre `val_data`:

In [4]:
random_classifier <- function(data) {
  classes <- numeric(length(data$prediction_prob))

  for (i in seq_along(data$prediction_prob)) {
    if (data$prediction_prob[i] > 0.5) {
      classes[i] <- 1
    } else {
      classes[i] <- 0
    }
  }
  data$prediction_class <- classes
  return(data)
}

val_df <- random_classifier(val_df)
head(val_df)

,id,especie,ultima_modificacion,altura,circ_tronco_cm,diametro_tronco,long,lat,seccion,nombre_seccion,area_seccion,inclinacion_peligrosa,prediction_prob,prediction_class
,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,37462,Morera,04/01/2011 12:01,Medio (4 - 8 mts),61,Grande,-68.85742,-32.87073,6,Residencial Norte,2694682,0,0.91005486,1
2,20115,Morera,09/08/2011 12:38,Alto (> 8 mts),97,Grande,-68.83363,-32.86650,4,Cuarta Oeste,1173333,1,0.81068005,1
3,8965,Morera,14/01/2011 10:27,Alto (> 8 mts),195,Grande,-68.84689,-32.86636,6,Residencial Norte,2694682,0,0.36127344,0
4,43061,Paraiso,03/10/2011 10:04,Medio (4 - 8 mts),100,Grande,-68.83222,-32.87843,4,Cuarta Este Área Fundacional,1822016,0,0.08436152,0
5,13030,Morera,16/02/2011 09:49,Medio (4 - 8 mts),64,Grande,-68.84500,-32.86599,4,Cuarta Oeste,1173333,0,0.86743940,1
6,37942,Acer,27/08/2010 10:50,Medio (4 - 8 mts),110,Grande,-68.84775,-32.87421,6,Residencial Norte,2694682,0,0.32003170,0


In [5]:
conf_matrix <- confusionMatrix(
  data = as.factor(val_df$prediction_class),
  reference = as.factor(val_df$inclinacion_peligrosa),
  positive = "1"
)

print(conf_matrix)

Confusion Matrix and Statistics

          Reference
Prediction    0    1
         0 2837  312
         1 2870  364
                                          
               Accuracy : 0.5015          
                 95% CI : (0.4891, 0.5138)
    No Information Rate : 0.8941          
    P-Value [Acc > NIR] : 1               
                                          
                  Kappa : 0.0133          
                                          
 Mcnemar's Test P-Value : <2e-16          
                                          
            Sensitivity : 0.53846         
            Specificity : 0.49711         
         Pos Pred Value : 0.11255         
         Neg Pred Value : 0.90092         
             Prevalence : 0.10591         
         Detection Rate : 0.05703         
   Detection Prevalence : 0.50666         
      Balanced Accuracy : 0.51779         
                                          
       'Positive' Class : 1               
                        

Luego, creamos la funcion `biggerclass_classifier` y aplicamos las operaciones correspondientes a nuestro set de validación:

In [6]:
biggerclass_classifier <- function(data) {
  majority <- ifelse(sum(data$inclinacion_peligrosa == 1) > sum(data$inclinacion_peligrosa == 0), 1, 0) # nolint: line_length_linter.
  data$prediction_class <- majority
  return(data)
}

val_df <- read_csv('https://raw.githubusercontent.com/gabichulas/ia-uncuyo-2024/refs/heads/main/tp7-ml/data/arbolado-mendoza-dataset-validation.csv', show_col_types = FALSE) # nolint: line_length_linter.
val_df <- biggerclass_classifier(val_df)

head(val_df)

conf_matrix <- confusionMatrix(
  data = as.factor(val_df$prediction_class),
  reference = as.factor(val_df$inclinacion_peligrosa),
  positive = "1"
)

print(conf_matrix)

id,especie,ultima_modificacion,altura,circ_tronco_cm,diametro_tronco,long,lat,seccion,nombre_seccion,area_seccion,inclinacion_peligrosa,prediction_class
<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>
37462,Morera,04/01/2011 12:01,Medio (4 - 8 mts),61,Grande,-68.85742,-32.87073,6,Residencial Norte,2694682,0,0
20115,Morera,09/08/2011 12:38,Alto (> 8 mts),97,Grande,-68.83363,-32.86650,4,Cuarta Oeste,1173333,1,0
8965,Morera,14/01/2011 10:27,Alto (> 8 mts),195,Grande,-68.84689,-32.86636,6,Residencial Norte,2694682,0,0
43061,Paraiso,03/10/2011 10:04,Medio (4 - 8 mts),100,Grande,-68.83222,-32.87843,4,Cuarta Este Área Fundacional,1822016,0,0
13030,Morera,16/02/2011 09:49,Medio (4 - 8 mts),64,Grande,-68.84500,-32.86599,4,Cuarta Oeste,1173333,0,0
37942,Acer,27/08/2010 10:50,Medio (4 - 8 mts),110,Grande,-68.84775,-32.87421,6,Residencial Norte,2694682,0,0


Warning message in confusionMatrix.default(data = as.factor(val_df$prediction_class), :
"Levels are not in the same order for reference and data. Refactoring data to match."


Confusion Matrix and Statistics

          Reference
Prediction    0    1
         0 5707  676
         1    0    0
                                          
               Accuracy : 0.8941          
                 95% CI : (0.8863, 0.9015)
    No Information Rate : 0.8941          
    P-Value [Acc > NIR] : 0.5102          
                                          
                  Kappa : 0               
                                          
 Mcnemar's Test P-Value : <2e-16          
                                          
            Sensitivity : 0.0000          
            Specificity : 1.0000          
         Pos Pred Value :    NaN          
         Neg Pred Value : 0.8941          
             Prevalence : 0.1059          
         Detection Rate : 0.0000          
   Detection Prevalence : 0.0000          
      Balanced Accuracy : 0.5000          
                                          
       'Positive' Class : 1               
                        

Creamos las funciones `create_folds` y `cross_validation`:

In [7]:
create_folds <- function(data, k) {
  set.seed(12)
  indices <- sample(seq_len(nrow(data)))
  folds <- split(indices, cut(seq_along(indices), breaks = k, labels = FALSE))
  names(folds) <- paste0("Fold", seq_len(k))
  return(folds)
}

cross_validation <- function(data, k) {
  folds <- create_folds(data, k)
  metrics <- data.frame(Accuracy = numeric(k), Precision = numeric(k), Sensitivity = numeric(k), Specificity = numeric(k)) 
  for (i in seq_along(folds)) {
    test_indices <- folds[[i]]
    train_indices <- setdiff(seq_len(nrow(data)), test_indices)
    
    train_data <- data[train_indices, ]
    test_data <- data[test_indices, ]
    
    model <- rpart(inclinacion_peligrosa ~ ., data = train_data, method = "class")
    predictions <- predict(model, test_data, type = "class")
    
    confusion <- confusionMatrix(predictions, as.factor(test_data$inclinacion_peligrosa), positive = "1")
    
    metrics$Accuracy[i] <- confusion$overall["Accuracy"]
    metrics$Precision[i] <- confusion$byClass["Pos Pred Value"]
    metrics$Sensitivity[i] <- confusion$byClass["Sensitivity"]
    metrics$Specificity[i] <- confusion$byClass["Specificity"]
  }
  
  results <- list(
    Accuracy = list(mean = mean(metrics$Accuracy), sd = sd(metrics$Accuracy)),
    Precision = list(mean = mean(metrics$Precision), sd = sd(metrics$Precision)),
    Sensitivity = list(mean = mean(metrics$Sensitivity), sd = sd(metrics$Sensitivity)),
    Specificity = list(mean = mean(metrics$Specificity), sd = sd(metrics$Specificity))
  )
  
  return(results)
}

